In [2]:
%run ../yp_utils.py

# Initial setup

In [3]:
paper_pmid = 23358190
paper_name = 'gaytan_vulpe_2013' 

In [4]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [5]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [25]:
original_data = pd.read_excel('raw_data/toxsci_12_0972_File010.xlsx', sheet_name='mutants with altered growth in ', skiprows=1)

In [26]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 749 x 6


In [27]:
original_data.head()

,ORF,25% IC20 (115 μM),50% IC20 (230 μM),100% IC20 (460 μM),Name,Description of deleted gene
0,YOL081W,-3.90,-4.95,-6.80,IRA2,GTPase-activating protein that negatively regu...
1,YPL120W,NaN,NaN,-6.25,VPS30,Subunit of phosphatidylinositol (PtdIns) 3-kin...
2,YOR070C,-4.00,-4.40,-6.10,GYP1,Cis-golgi GTPase-activating protein (GAP) for ...
3,YBR068C,-5.60,NaN,-5.65,BAP2,"High-affinity leucine permease, functions as a..."
4,YLR044C,-3.75,-4.85,-5.60,PDC1,Major of three pyruvate decarboxylase isozymes...


In [28]:
original_data['orf'] = original_data['ORF'].astype(str)

In [29]:
# Eliminate all white spaces & capitalize
original_data['orf'] = clean_orf(original_data['orf'])

In [30]:
# Translate to ORFs 
original_data['orf'] = translate_sc(original_data['orf'], to='orf')

In [31]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf'])
print(original_data.loc[~t,])

             ORF  25% IC20 (115 μM)  50% IC20 (230 μM)  100% IC20 (460 μM)  \
index_input                                                                  
427          NaN                NaN                NaN                 NaN   

            Name Description of deleted gene  orf  
index_input                                        
427          NaN                         NaN  NAN  


In [32]:
original_data = original_data.loc[t,:]

In [33]:
original_data.set_index('orf', inplace=True)

In [34]:
original_data = original_data[['25% IC20 (115 μM)','50% IC20 (230 μM)','100% IC20 (460 μM)']].copy()

In [35]:
original_data[original_data.isnull()] = 0

In [36]:
original_data = original_data.groupby(original_data.index).mean()

In [37]:
original_data.shape

(748, 3)

# Prepare the final dataset

In [38]:
data = original_data.copy()

In [39]:
dataset_ids = [208, 207, 206]
datasets = datasets.reindex(index=dataset_ids)

In [40]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [41]:
data.head()

dataset_id,208,207,206
data_type,value,value,value
orf,,,
YAL002W,2.3,2.6,0.0
YAL020C,1.6,0.0,0.0
YAL044C,-2.0,-2.8,-3.6
YAL053W,0.0,0.0,-1.5
YAL056W,0.0,-2.2,-3.1


## Subset to the genes currently in SGD

In [42]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 3


In [43]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,208,207,206
,data_type,value,value,value
gene_id,orf,,,
2,YAL002W,2.3,2.6,0.0
18,YAL020C,1.6,0.0,0.0
42,YAL044C,-2.0,-2.8,-3.6
49,YAL053W,0.0,0.0,-1.5
52,YAL056W,0.0,-2.2,-3.1


# Normalize

In [44]:
data_norm = normalize_phenotypic_scores(data, has_tested=False)

In [45]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [46]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

,dataset_id,208,207,206,208,207,206
,data_type,value,value,value,valuez,valuez,valuez
gene_id,orf,,,,,,
2,YAL002W,2.3,2.6,0.0,3.132805,3.633095,0.039506
18,YAL020C,1.6,0.0,0.0,2.164026,0.030789,0.039506
42,YAL044C,-2.0,-2.8,-3.6,-2.818267,-3.848618,-3.871603
49,YAL053W,0.0,0.0,-1.5,-0.050326,0.030789,-1.590123
52,YAL056W,0.0,-2.2,-3.1,-0.050326,-3.017316,-3.328393


# Print out

In [47]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [48]:
from IO.save_data_to_db3 import *

In [49]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/3 [00:00<?, ?it/s]

Deleting all datasets for PMID 23358190...
Inserting the new data...


100%|██████████| 3/3 [00:03<00:00,  1.26s/it]

Updating the data_modified_on field...
